# Lake model continued - sensitivity analysis

In the previous week you used the lake problem as a means of getting acquainted with the workbench. In this assignment we will continue with the lake problem, focussing explicitly on using it for open exploration. You can use the second part of the [open exploration tutorial](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html) for help.

**It is paramount that you are using the lake problem with 100 decision variables, rather than the one found on the website with the separate anthropogenic release decision**

There is substantial support in the ema_workbench for global sensitivity. For this, the workbench relies on [SALib](https://salib.readthedocs.io/en/latest/) and feature scoring which is a machine learning alternative for global sensitivity analysis.

## 1. SOBOL
1. Apply Sobol with 3 separate release policies (0, 0.05, and 0.1) and analyse the results for each release policy separately focusing on the reliability objective. Do the sensitivities change depending on the release policy? Can you explain why or why not?

*hint: you can use sobol sampling for the uncertainties, and set policies to a list with the 3 different release policies. Next, for the analysis using logical indexing on the experiment.policy column you can select the results for each separate release policy and apply sobol to each of the three separate release policies. If this sounds too complicated, just do it on each release policy separately.*

In [6]:
from ema_workbench import Samplers
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, ScalarOutcome, perform_experiments, Constant, ema_logging, MultiprocessingEvaluator)

from ema_workbench import Samplers

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

# Import the lake model function from the specific Python file
from lakemodel_function import lake_problem

In [8]:
# Set up EMA Workbench logging
ema_logging.log_to_stderr(ema_logging.INFO)

# Create a model instance
model = Model('lakeproblem', function=lake_problem)

# Define uncertainties directly within the model
model.uncertainties = [
    RealParameter("mean", 0.01, 0.05),
    RealParameter("stdev", 0.001, 0.005),
    RealParameter("b", 0.1, 0.45),
    RealParameter("q", 2, 4.5),
    RealParameter("delta", 0.93, 0.99)
]

# Define outcomes
model.outcomes = [
    ScalarOutcome('max_P'),
    ScalarOutcome('utility'),
    ScalarOutcome('inertia'),
    ScalarOutcome('reliability')
]

In [9]:
from ema_workbench import Policy

# Define release policies along with decision levers
release_policies = {
    "Release Policy 0": {f'l{i}': 0 for i in range(100)},
    "Release Policy 0.05": {f'l{i}': 0.05 for i in range(100)},
    "Release Policy 0.1": {f'l{i}': 0.1 for i in range(100)}
}

# Run Sobol experiments for each release policy separately
with MultiprocessingEvaluator(model) as evaluator:
    experiments = []
    outcomes = []
    
    for policy_name, policy_levers in release_policies.items():
        policy = Policy(policy_name, **policy_levers)
        exp, out = evaluator.perform_experiments(scenarios=1000, policies=[policy])
        experiments.append(exp)
        outcomes.append(out)

[MainProcess/INFO] pool started with 20 workers
[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
100%|█████████████████████████████████████| 1000/1000 [00:02<00:00, 356.90it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 1566.15it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 2076.04it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [11]:
from ema_workbench.analysis import feature_scoring

# Perform Sobol sensitivity analysis for each release policy separately
for i, (policy_name, out) in enumerate(zip(release_policies.keys(), outcomes)):
    print(f"Running Sobol analysis for {policy_name}...")
    sobol_indices = feature_scoring.get_SALib_sobol_indices(model, out, 
                                                            alg_name='Sobol', 
                                                            nr_resamples=100)
    # Print the Sobol indices for reliability outcome
    print(f"Sobol indices for reliability (Policy {i+1}): {sobol_indices['reliability']}")

Running Sobol analysis for Release Policy 0...


AttributeError: module 'ema_workbench.analysis.feature_scoring' has no attribute 'get_SALib_sobol_indices'

## 2. Feature scoring
Repeat the above analysis for the 3 release policies but now with extra trees [feature scoring](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html) and for all outcomes of interest. As a bonus, use the sobol experiment results as input for extra trees, and compare the results with those resulting from latin hypercube sampling.

*hint: you can use [seaborn heatmaps](https://seaborn.pydata.org/generated/seaborn.heatmap.html) for a nice figure of the results. See also the [features scoring](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html#feature-scoring) section of the tutorial.*
